In [1]:
import os, torch
from pypdf import PdfReader
import chromadb
from sentence_transformers import SentenceTransformer

from docx import Document
from pptx import Presentation


# ================================
# 1. Load file (PDF, DOCX, PPTX)
# ================================
def load_file_pages(path):
    ext = os.path.splitext(path)[1].lower()

    # ========== PDF ==========
    if ext == ".pdf":
        reader = PdfReader(path)
        pages = []
        for page in reader.pages:
            text = page.extract_text() or ""
            pages.append(text)
        return pages

    # ========== DOCX ==========
    elif ext == ".docx":
        doc = Document(path)
        pages = []

        buffer = []
        paragraph_count = 0

        for para in doc.paragraphs:
            text = para.text.strip()
            if text:
                buffer.append(text)
                paragraph_count += 1

            # Cho thành "page" sau mỗi 20 đoạn (tùy chỉnh)
            if paragraph_count >= 20:
                pages.append("\n".join(buffer))
                buffer = []
                paragraph_count = 0

        if buffer:
            pages.append("\n".join(buffer))

        return pages

    # ========== PPTX ==========
    elif ext == ".pptx":
        pres = Presentation(path)
        pages = []

        for slide in pres.slides:
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    slide_text.append(shape.text)
            pages.append("\n".join(slide_text))

        return pages

    # ========== Không hỗ trợ ==========
    else:
        raise ValueError("Unsupported file format: only PDF, DOCX, PPTX.")



# ================================
# 2. Chunk 1 trang
# ================================
def chunk_page(text, chunk_size=800, overlap=200):
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = end - overlap

    return chunks


# ================================
# 3. Build chunks + metadata
# ================================
def build_chunks(path):
    pages = load_file_pages(path)

    all_chunks = []
    all_ids = []
    all_meta = []

    for page_idx, text in enumerate(pages):
        page_number = page_idx + 1
        chunks = chunk_page(text)

        for ci, c in enumerate(chunks):
            all_chunks.append(c)
            all_ids.append(f"{os.path.basename(path)}_p{page_number}_c{ci}")
            all_meta.append({
                "page": page_number,
                "chunk": ci
            })

    return all_chunks, all_ids, all_meta, pages



# ================================
# 4. Model + Vector DB
# ================================
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

chroma = chromadb.Client()
collection = chroma.get_or_create_collection(
    name="pdf_docs",
    metadata={"hnsw:space": "cosine"}
)



# ================================
# 5. Index PDF/DOCX/PPTX
# ================================
def index_pdf(path, batch_size=2):
    chunks, ids, metadata, pages = build_chunks(path)

    all_embeds = []

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i : i + batch_size]

        with torch.no_grad():
            vec = model.encode(batch).tolist()

        all_embeds.extend(vec)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        print(f"Đã embed {i + len(batch)}/{len(chunks)} chunks", end="\r")

    collection.add(
        ids=ids,
        documents=chunks,
        metadatas=metadata,
        embeddings=all_embeds
    )

    print(f"\nIndexed {len(chunks)} chunks từ file {path}")
    return pages



# ================================
# 6. Lấy trang lân cận
# ================================
def get_surrounding_pages(page, pages):
    prev_page = pages[page - 2] if page > 1 else None
    this_page = pages[page - 1]
    next_page = pages[page] if page < len(pages) else None

    return prev_page, this_page, next_page



# ============================================================
# 7. SEARCH 2 BƯỚC: TopK → Expand → Re-chunk → Rerank
# ============================================================
def search(query, pages, top_k_first=3, top_k_second=5, chunk_size=800, overlap=200):

    # STEP 1 ───────────────────────────────────────────────
    q_emb = model.encode([query]).tolist()

    result = collection.query(
        query_embeddings=q_emb,
        n_results=top_k_first,
        include=["documents", "metadatas"]
    )

    metas = result["metadatas"][0]

    # STEP 2 ───────────────────────────────────────────────
    expanded_pages = []
    seen = set()

    for meta in metas:
        page = meta["page"]
        prev_page, this_page, next_page = get_surrounding_pages(page, pages)
        candidates = [prev_page, this_page, next_page]

        for p in candidates:
            if p and p not in seen:
                expanded_pages.append(p)
                seen.add(p)

    # STEP 3 ───────────────────────────────────────────────
    big_text = "\n\n".join(expanded_pages)
    words = big_text.split()

    re_chunks = []
    start = 0
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        re_chunks.append(chunk)
        start = end - overlap

    # STEP 4 ───────────────────────────────────────────────
    embeds = model.encode(re_chunks).tolist()

    temp = chromadb.Client().create_collection(
        name="temp_rerank",
        metadata={"hnsw:space": "cosine"},
        get_or_create=True
    )

    temp_ids = [f"rechunk_{i}" for i in range(len(re_chunks))]
    temp.add(ids=temp_ids, embeddings=embeds, documents=re_chunks)

    result2 = temp.query(
        query_embeddings=q_emb,
        n_results=top_k_second,
        include=["documents", "distances"]
    )

    docs2 = result2["documents"][0]
    dists2 = result2["distances"][0]

    # STEP 5 ───────────────────────────────────────────────
    output = []
    for i, (doc, dist) in enumerate(zip(docs2, dists2), start=1):
        output.append({
            "rank": i,
            "matched_chunk": doc,
            "score": 1 - dist,
            "raw_distance": dist
        })

    return output



# # ============================================================
# # 8. RUN
# # ============================================================
# if __name__ == "__main__":
#     pages = index_pdf("test.docx")   # hoặc test.pdf, test.pptx

#     results = search("Mô hình nào được dùng?", pages)

#     for r in results:
#         print("="*80)
#         print("Rank:", r["rank"])
#         print("Score:", r["score"])
#         print(r["matched_chunk"])


In [2]:
pages = index_pdf("test2.docx")

Đã embed 20/20 chunks
Indexed 20 chunks từ file test2.docx


In [5]:
results = search("Điều 20. Trách nhiệm triển khai Quy chế", pages, top_k_first=3, top_k_second=5, chunk_size=300, overlap=100)

for r in results:
    print("="*80)
    print("Thông tin", r["rank"], ":")
    print("Uy tín", r["score"],":")
    print(r["matched_chunk"])

Thông tin 1 :
Uy tín 0.41750895977020264 :
chế này; các quy định của Luật Bảo vệ bí mật nhà nước và các văn bản hướng dẫn thi hành; các quy định về bảo vệ bí mật của Bộ Thông tin và Truyền thông và Ủy ban quản lý vốn nhà nước tại doanh nghiệp. 1. Tổng giám đốc có trách nhiệm hướng dẫn, đôn đốc, kiểm tra việc thực hiện Quy chế này tại các Ban, Văn phòng, Phòng thuộc khối Cơ quan Tổng công ty và các đơn vị trực thuộc Tổng công ty. 2. Trưởng các Ban, Văn phòng, Phòng thuộc khối Cơ quan Tổng công ty, Giám đốc các đơn vị trực thuộc Tổng công ty có trách nhiệm tổ chức thực hiện và phổ biến Quy chế này đến cán bộ, nhân viên thuộc quyền quản lý để thực hiện. 3. Ban Công nghệ thông tin Tổng công ty có trách nhiệm chủ trì rà soát và hướng dẫn, quy định việc mã hoá, các biện pháp đảm bảo tính bảo mật trong trường hợp sử dụng máy tính hoặc các thiết bị khác, thư điện tử, hệ thống E-Office của Tổng công ty để xử lý, lưu trữ các thông tin, tài liệu mật cơ quan. Nguyễn Thị Hiếu hieu.nth@mobifone.vn h

In [2]:
!python -m spacy download en_core_web_md


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/33.5 MB ? eta -:--:--
     ---------------------------------------- 0.3/33.5 MB ? eta -:--:--
     -- ------------------------------------- 1.8/33.5 MB 6.7 MB/s eta 0:00:05
     ---- ----------------------------------- 3.7/33.5 MB 7.3 MB/s eta 0:00:05
     ----- ---------------------------------- 4.5/33.5 MB 6.2 MB/s eta 0:00:05
     ------ --------------------------------- 5.5/33.5 MB 6.0 MB/s eta 0:00:05
     ------- -------------------------------- 6.6/33.5 MB 6.3 MB/s eta 0:00:05
     -------- ------------------------------- 7.1/33.5 MB 5.5 MB/s eta 0:00:05
     --------- ------------------------------ 7.9/33.5 MB 5.2 MB/s eta 0:00:05
     ---------- ----------------------------- 9.2/33.5 MB 5.2 MB/s eta 0:00:05
     ----------- ---------------------------- 9.4/33.5 MB 5.2 MB/s eta 0:00:05
     ------------ --------------------------- 10.2/33.5 MB 4.7 MB/s

In [ ]:
import os, re, gc, torch
from collections import Counter

from pypdf import PdfReader
from docx import Document
from pptx import Presentation

import chromadb
from sentence_transformers import SentenceTransformer

import spacy
from transformers import pipeline


# =====================================================
# 0. LOAD MODELS (CHỈ 1 LẦN)
# =====================================================

device = "cuda" if torch.cuda.is_available() else "cpu"

embed_model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    device=device
)

ner_multi = pipeline(
    "ner",
    model="Babelscape/wikineural-multilingual-ner",
    grouped_entities=True,
    device=0 if device == "cuda" else -1
)

email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_regex = r"\+?\d[\d\- ]{7,}\d"


# =====================================================
# 1. LOAD FILE → PAGES
# =====================================================

def load_file_pages(path):
    ext = os.path.splitext(path)[1].lower()

    if ext == ".pdf":
        reader = PdfReader(path)
        return [p.extract_text() or "" for p in reader.pages]

    if ext == ".docx":
        doc = Document(path)
        pages, buf, cnt = [], [], 0
        for para in doc.paragraphs:
            if para.text.strip():
                buf.append(para.text.strip())
                cnt += 1
            if cnt >= 20:
                pages.append("\n".join(buf))
                buf, cnt = [], 0
        if buf:
            pages.append("\n".join(buf))
        return pages

    if ext == ".pptx":
        pres = Presentation(path)
        pages = []
        for slide in pres.slides:
            texts = []
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    texts.append(shape.text)
            pages.append("\n".join(texts))
        return pages

    raise ValueError("Unsupported file format")


# =====================================================
# 2. PAGE → CHUNK
# =====================================================

def chunk_page(text, size=800, overlap=200):
    words = text.split()
    chunks, i = [], 0
    while i < len(words):
        chunks.append(" ".join(words[i:i + size]))
        i += size - overlap
    return chunks


def build_chunks(path):
    pages = load_file_pages(path)

    chunks, ids, metas = [], [], []

    for p_idx, text in enumerate(pages):
        for c_idx, c in enumerate(chunk_page(text)):
            chunks.append(c)
            ids.append(f"{os.path.basename(path)}_p{p_idx}_c{c_idx}")
            metas.append({"page": p_idx})

    return chunks, ids, metas, pages


# =====================================================
# 3. VECTOR DATABASE
# =====================================================

chroma = chromadb.Client()
collection = chroma.get_or_create_collection(
    name="pdf_docs",
    metadata={"hnsw:space": "cosine"}
)


def index_file(path, batch_size=8):
    chunks, ids, metas, pages = build_chunks(path)

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        with torch.no_grad():
            emb = embed_model.encode(batch).tolist()

        collection.add(
            ids=ids[i:i + batch_size],
            documents=batch,
            metadatas=metas[i:i + batch_size],
            embeddings=emb
        )

        if device == "cuda":
            torch.cuda.empty_cache()

    return pages


# =====================================================
# 4. ENTITY EXTRACTION + HIGHLIGHT
# =====================================================

def extract_entities(text, max_len=1200):
    text = text[:max_len]
    entities = set()

    for ent in ner_multi(text):
        entities.add(ent["word"])

    entities.update(re.findall(email_regex, text))
    entities.update(re.findall(phone_regex, text))

    return sorted(entities, key=len, reverse=True)


def highlight_markdown(text, entities):
    for e in entities:
        if not e.strip():
            continue
        text = re.sub(
            rf"(?<!\*)({re.escape(e)})(?!\*)",
            r"**\1**",
            text
        )
    return text


# =====================================================
# 5. PAGE EMBEDDING CACHE + PAGE SIMILARITY
# =====================================================

PAGE_EMB_CACHE = {}


def get_page_embedding(pid, pages):
    if pid not in PAGE_EMB_CACHE:
        with torch.no_grad():
            PAGE_EMB_CACHE[pid] = embed_model.encode(
                pages[pid],
                normalize_embeddings=True
            )
    return PAGE_EMB_CACHE[pid]


def find_related_pages(target_pid, pages, top_k=2):
    target_emb = get_page_embedding(target_pid, pages)

    scores = []

    for pid in range(len(pages)):
        if pid == target_pid:
            continue

        emb = get_page_embedding(pid, pages)
        sim = float(
            torch.dot(
                torch.tensor(target_emb),
                torch.tensor(emb)
            )
        )
        scores.append((pid, sim))

    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]


# =====================================================
# 6. SEARCH
# =====================================================

def search(query, pages, chunk_topk=10, page_topk=3):

    with torch.no_grad():
        q_emb = embed_model.encode([query]).tolist()

    result = collection.query(
        query_embeddings=q_emb,
        n_results=chunk_topk,
        include=["metadatas"]
    )

    page_counter = Counter(
        m["page"] for m in result["metadatas"][0]
    )

    top_pages = [
        pid for pid, _ in page_counter.most_common(page_topk)
    ]

    outputs = []

    for pid in top_pages:
        page_text = pages[pid]
        entities = extract_entities(page_text)
        highlighted = highlight_markdown(page_text, entities)

        related = find_related_pages(pid, pages, top_k=2)

        related_pages = []
        for r_pid, score in related:
            r_text = pages[r_pid]
            r_entities = extract_entities(r_text)
            r_highlight = highlight_markdown(r_text, r_entities)

            related_pages.append({
                "page": r_pid + 1,
                "score": round(score, 4),
                "highlighted_text": r_highlight
            })

        outputs.append({
            "page": pid + 1,
            "highlighted_text": highlighted,
            "related_pages": related_pages
        })

    gc.collect()
    if device == "cuda":
        torch.cuda.empty_cache()

    return outputs


# =====================================================
# 7. MAIN
# =====================================================

# if __name__ == "__main__":
#     pages = index_file("test3.pdf")

#     results = search(
#         query="Quy định chuyển tiếp",
#         pages=pages,
#         chunk_topk=10,
#         page_topk=3
#     )

#     for r in results:
#         print("=" * 100)
#         print(f"PAGE {r['page']}")
#         print("-" * 100)
#         print(r["highlighted_text"])
#         print("\nRELATED PAGES:")
#         for rp in r["related_pages"]:
#             print(f"  -> Page {rp['page']} | score={rp['score']}")


Device set to use cuda:0
C:\Users\MSI\AppData\Roaming\Python\Python313\site-packages\transformers\pipelines\token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [5]:
pages = index_file("test2.docx")


In [6]:
results = search(
    query="Điều 10. Thống kê, lưu trữ tài liệu, vật chứa bí mật",
    pages=pages,
    chunk_topk=10,
    page_topk=3
)

for r in results:
    print("=" * 100)
    print(f"PAGE {r['page']}")
    print("-" * 100)
    print(r["highlighted_text"])
    print("\nRELATED PAGES:")
    for rp in r["related_pages"]:
        print(f"  -> Page {rp['page']} | score={rp['score']} | text={rp['highlighted_text']}")

PAGE 5
----------------------------------------------------------------------------------------------------
a. Vận chuyển, gửi tài liệu, vật chứa bí mật do người làm công tác liên quan đến bí mật hoặc văn thư của cơ quan, đơn vị thực hiện. Trong quá trình vận chuyển tài liệu, vật chứa bí mật phải có biện pháp bảo đảm an toàn, trường hợp cần thiết phải có lực
lượng bảo vệ. Vận chuyển tài liệu, vật chứa bí mật qua dịch vụ bưu chính được thực hiện theo quy định của pháp luật về bưu chính.
b.  Nơi gửi phải có các biện  pháp quản lý tài liệu, vật chứa bí  mật nhằm tránh những sai sót, mất mát. Tài liệu, vật chứa bí mật khi chuyển giao, gửi đi phải bỏ vào bì, không được gửi chung trong một bì với tài liệu thường. Bì gửi tài liệu, vật chứa bí mật phải làm bằng giấy có chất lượng tốt, độ thấm nước thấp, chắc chắn, khó bóc, không nhìn thấu qua được, bì phải được dán kín.
Trường  hợp  tài  liệu,  vật  chứa  bí  mật  thuộc  độ  'Tuyệt  mật'  phải  được  bảo  vệ bằng hai lớp phong bì: Bì trong ghi

In [ ]:
import os, re, gc, torch
from collections import Counter
from typing import List

from pypdf import PdfReader
from docx import Document
from pptx import Presentation

import chromadb
from chromadb.config import Settings

from sentence_transformers import SentenceTransformer
from transformers import pipeline

# =====================================================
# 0. CONFIG
# =====================================================

CHROMA_PATH = "./chroma_db"
os.makedirs(CHROMA_PATH, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"

embed_model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    device=device
)

ner_multi = pipeline(
    "ner",
    model="Babelscape/wikineural-multilingual-ner",
    grouped_entities=True,
    device=0 if device == "cuda" else -1
)

email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_regex = r"\+?\d[\d\- ]{7,}\d"

# =====================================================
# 1. INIT CHROMA (PERSISTENT)
# =====================================================

chroma = chromadb.Client(
    Settings(
        persist_directory=CHROMA_PATH,
        anonymized_telemetry=False
    )
)

# =====================================================
# 2. FILE → PAGES
# =====================================================

def load_file_pages(path: str) -> List[str]:
    ext = os.path.splitext(path)[1].lower()

    if ext == ".pdf":
        reader = PdfReader(path)
        return [p.extract_text() or "" for p in reader.pages]

    if ext == ".docx":
        doc = Document(path)
        pages, buf, cnt = [], [], 0
        for para in doc.paragraphs:
            if para.text.strip():
                buf.append(para.text.strip())
                cnt += 1
            if cnt >= 20:
                pages.append("\n".join(buf))
                buf, cnt = [], 0
        if buf:
            pages.append("\n".join(buf))
        return pages

    if ext == ".pptx":
        pres = Presentation(path)
        pages = []
        for slide in pres.slides:
            texts = []
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    texts.append(shape.text)
            pages.append("\n".join(texts))
        return pages

    raise ValueError("Unsupported file")

# =====================================================
# 3. PAGE → CHUNK
# =====================================================

def chunk_page(text, size=800, overlap=200):
    words = text.split()
    chunks, i = [], 0
    while i < len(words):
        chunks.append(" ".join(words[i:i + size]))
        i += size - overlap
    return chunks

# =====================================================
# 4. UPLOAD FILE → COLLECTION
# =====================================================

def upload_file(file_path: str):
    name = f"doc_{os.path.basename(file_path)}"
    pages = load_file_pages(file_path)

    col = chroma.get_or_create_collection(
        name=name,
        metadata={"space": "cosine"}
    )

    # ===== PAGE LEVEL =====
    with torch.no_grad():
        page_embs = embed_model.encode(
            pages,
            normalize_embeddings=True
        ).tolist()

    col.add(
        ids=[f"page_{i}" for i in range(len(pages))],
        documents=pages,
        embeddings=page_embs,
        metadatas=[{"type": "page", "page": i} for i in range(len(pages))]
    )

    # ===== CHUNK LEVEL =====
    for pid, text in enumerate(pages):
        chunks = chunk_page(text)
        with torch.no_grad():
            embs = embed_model.encode(chunks).tolist()

        col.add(
            ids=[f"chunk_{pid}_{i}" for i in range(len(chunks))],
            documents=chunks,
            embeddings=embs,
            metadatas=[
                {"type": "chunk", "page": pid} for _ in chunks
            ]
        )

    chroma.persist()
    return name

# =====================================================
# 5. LIST COLLECTIONS
# =====================================================

def list_collections():
    return [c.name for c in chroma.list_collections()]

# =====================================================
# 6. ENTITY + HIGHLIGHT
# =====================================================

def extract_entities(text, max_len=1200):
    text = text[:max_len]
    ents = set()

    for e in ner_multi(text):
        ents.add(e["word"])

    ents.update(re.findall(email_regex, text))
    ents.update(re.findall(phone_regex, text))

    return sorted(ents, key=len, reverse=True)

def highlight_markdown(text, entities):
    for e in entities:
        if not e.strip():
            continue
        text = re.sub(
            rf"(?<!\*)({re.escape(e)})(?!\*)",
            r"**\1**",
            text
        )
    return text

# =====================================================
# 7. PAGE SIMILARITY
# =====================================================

def find_related_pages(collection, target_page, top_k=2):
    target = collection.get(
        where={
            "$and": [
                {"type": "page"},
                {"page": target_page}
            ]
        },
        include=["embeddings"]
    )["embeddings"][0]

    pages = collection.get(
        where={"type": "page"},
        include=["embeddings", "metadatas"]
    )

    scores = []
    for emb, meta in zip(pages["embeddings"], pages["metadatas"]):
        pid = meta["page"]
        if pid == target_page:
            continue
        sim = float(torch.dot(
            torch.tensor(target),
            torch.tensor(emb)
        ))
        scores.append((pid, sim))

    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_k]

# =====================================================
# 8. SEARCH
# =====================================================

def search(collection_name, query, chunk_topk=10, page_topk=3):
    col = chroma.get_collection(collection_name)

    with torch.no_grad():
        q_emb = embed_model.encode([query]).tolist()

    res = col.query(
        query_embeddings=q_emb,
        n_results=chunk_topk,
        where={"type": "chunk"},
        include=["metadatas"]
    )

    page_counter = Counter(
        m["page"] for m in res["metadatas"][0]
    )

    top_pages = [
        pid for pid, _ in page_counter.most_common(page_topk)
    ]

    outputs = []

    for pid in top_pages:
        page_text = col.get(
            where={
                "$and": [
                    {"type": "page"},
                    {"page": pid}
                ]
            },
            include=["documents"]
        )["documents"][0]

        ents = extract_entities(page_text)
        highlight = highlight_markdown(page_text, ents)

        related_pages = []
        for r_pid, score in find_related_pages(col, pid):
            r_text = col.get(
                where={
                    "$and": [
                        {"type": "page"},
                        {"page": r_pid}
                    ]
                },
                include=["documents"]
            )["documents"][0]

            r_ents = extract_entities(r_text)
            r_high = highlight_markdown(r_text, r_ents)

            related_pages.append({
                "page": r_pid + 1,
                "score": round(score, 4),
                "highlighted_text": r_high
            })

        outputs.append({
            "page": pid + 1,
            "highlighted_text": highlight,
            "related_pages": related_pages
        })

    gc.collect()
    if device == "cuda":
        torch.cuda.empty_cache()

    return outputs


Device set to use cuda:0
C:\Users\MSI\AppData\Roaming\Python\Python313\site-packages\transformers\pipelines\token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [ ]:
upload_file("test3.pdf")

'test3.pdf'

In [ ]:
upload_file("test2.docx")

In [ ]:
upload_file("test.pdf")

'doc_test.pdf'

In [ ]:
# List
print(list_collections())

# Search
results = search(
    collection_name="",
    query="Quy định chuyển tiếp"
)

for r in results:
    print("=" * 80)
    print("PAGE", r["page"])
    print(r["highlighted_text"])


Collections: ['doc_test.pdf', 'doc_test2.docx', 'doc_test3.pdf']
PAGE: 1
| 1 | **Nguyễn Mạnh Thắng** | Chủ tịch HĐTV - Hội đồng thành viên | 10/11/2020 13:56:09 | - | | 2 | **Nguyễn Thị Phương Hạnh** | Phó Chánh văn phòng - Văn phòng | 10/11/2020 10:58:17 | VP kính gửi Chủ tịch HĐTV các Phiếu ý kiến đồng ý của Thành viên HĐTV và kính trình Chủ tịch ký ban hành **Quy chế Bảo vệ bí mật** tại TCT viễn **thông MobiFone**. Quy chế sẽ có hiệu lực áp dụng từ 01/01/2021 để đảm bảo các đơn vị triển khai các công tác chuẩn bị về mặt hệ thống. 26/01/2021 11:16:46 | TỔNG CÔNG TY VIỄN THÔNG MOBIFONE Số: 2110/QĐ-HĐTV CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc **Hà Nội**, ngày 10 tháng 11 năm 2020 QUYẾT ĐỊNH Ban hành Quy chế bảo vệ bí mật tại **Tổng công ty Viễn thông MobiFone** HỘI ĐỒNG THÀNH VIÊN TỔNG CÔNG TY VIỄN THÔNG MOBIFONE Căn cứ Quyết định số 1798/QĐ-BTTTT ngày 01/12/2014 của **Bộ Thông **tin** và Truyền thông** về việc thành lập **Tổng công ty Viễn thông MobiFone** trên 